In [ ]:
public void SummonEnd(UserMagic magic, Map map, Point location, MonsterInfo info)
{
    if (info == null || Dead) return;

    MonsterObject ob = Pets.FirstOrDefault(x => x.MonsterInfo == info);
    //拉狗
    if (ob != null)
    {
        //拉 不能重叠
        ob.PetRecall(false);
        return;
    }
    //招 可以重叠
    if (Pets.Count >= 2) return;//限制两个宝宝

    ob = MonsterObject.GetMonster(info);

    ob.PetOwner = this;
    ob.Owner = this;
    ob.OwnerID = Character.Index;
    
    PetAdd(ob);

    //SEnvir.Pets.Add(ob);

    ob.Master?.MinionList.Remove(ob);
    ob.Master = null;
    ob.Magics.Add(magic);
    // 宠物等级
    ob.SummonLevel = magic.Level;
    ob.SummonBaseLevel = magic.Level;
    ob.TameTime = SEnvir.Now.AddDays(365);
    ob.SummonSource = magic.Info.Magic;
    if (Buffs.Any(x => x.Type == BuffType.StrengthOfFaith))
        ob.Magics.Add(Magics[MagicType.StrengthOfFaith]);


    if (magic.Info.Magic == MagicType.SummonDemonicCreature && Magics.TryGetValue(MagicType.DemonicRecovery, out UserMagic demonRecovery))
        ob.Magics.Add(demonRecovery);

    //第一次招狗
    Cell cell = map.GetCell(location);
    if (cell == null)
    {
        //召唤 墙随机 
        if (SEnvir.Random.Next(100) > 40)
        {
            Rectangle rect = Functions.GetDirectionRange(CurrentLocation, Direction, Config.MaxViewRange + 3);
            cell = map.GetCell(map.GetRandomLocation(rect.Left,rect.Right,rect.Top,rect.Bottom));
        }
        else
        {
            cell = map.GetCell(map.GetRandomLocation(location, Config.MaxViewRange + 3));
        }
    }
    //否则可以重叠
    
    if (cell == null || cell.Movements != null || !ob.Spawn(map, cell.Location))
        ob.Spawn(CurrentMap, CurrentLocation);
    
    // 添加宠物buf
    RefreshPetBuff(ob,true);

    // 召唤生物满血量          
    ob.SetHP(ob.Stats[Stat.Health]);

    LevelMagic(magic);
}